<a href="https://colab.research.google.com/github/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/blob/main/Opt_In_VariacionEn_Demanda_y_LeadTima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres de ITEM
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # Nivel de servicio 95%
R_dias = 0.5  # Tiempo de reorden en días
MIN_DATOS_PARA_CALCULO = 18  # Usamos los últimos 18 meses

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # Extraer ventas (columnas de mes)
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # --- 🔧 Imputar valores negativos ---
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0

    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue
        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    ventas_imp = ventas_imp.clip(lower=0)

    # --- 📊 Últimos 18 meses para demanda ---
    ultimos_18 = ventas_imp[-18:]

    if len(ultimos_18.dropna()) < MIN_DATOS_PARA_CALCULO:
        resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'SS_Techo_Dias': np.nan,
            'SS_Optimo_Dias': np.nan
        })
        continue

    # Promedio y desviación de demanda (en unidades/mes)
    mu_d = ultimos_18.mean()
    sigma_d = ultimos_18.std()

    # Si no hay variabilidad, asignar un mínimo
    if pd.isna(sigma_d) or sigma_d == 0:
        sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

    # --- Lead Time en días ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_values = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_values) > 0:
            mu_L = lt_values.mean()  # Promedio de lead time en días
            sigma_L = lt_values.std()  # Desviación de lead time en días
        else:
            mu_L = 5.0
            sigma_L = 0.5
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando L = 5 días, σL = 0.5.")
        mu_L = 5.0
        sigma_L = 0.5

    # --- Fórmula completa de Stock de Seguridad en UNIDADES ---
    # Ss = Z * sqrt((μ_L + R)*σ_d² + σ_L²*μ_d²)
    try:
        SS_unidades = Z * np.sqrt(
            (mu_L + R_dias) * sigma_d**2 +
            sigma_L**2 * mu_d**2
        )
    except:
        SS_unidades = 0.1

    # Convertir SS a días
    if mu_d > 0:
        SS_dias = (SS_unidades / mu_d) * 30  # Ajuste a días (por mes de 30 días)
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    # --- Stock de Seguridad Techo en Días ---
    SS_techo_dias = SS_dias + mu_L + R_dias
    SS_techo_dias = round(SS_techo_dias, 2)

    # --- Stock Óptimo en Días ---
    SS_optimo_dias = SS_dias + (mu_L + R_dias) / 2
    SS_optimo_dias = round(SS_optimo_dias, 2)

    # Agregar al resultado
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'SS_Techo_Dias': SS_techo_dias,
        'SS_Optimo_Dias': SS_optimo_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Resultados con fórmula mejorada (variabilidad en demanda y lead time):")
print(resultado_df)

# Opcional: guardar
# resultado_df.to_excel("Stock_Seguridad_Final_Mejorado.xlsx", index=False)

In [1]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # 95% nivel de servicio
R_dias = 0.5  # Tiempo de reorden en días
R_meses = R_dias / 30  # Convertir a meses
MIN_DATOS_PARA_CALCULO = 18

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # --- 🔹 Extraer y limpiar demanda ---
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # Imputar negativos
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0

    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue
        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    ventas_imp = ventas_imp.clip(lower=0)

    # Últimos 18 meses
    ultimos_18 = ventas_imp[-18:]
    if len(ultimos_18.dropna()) < MIN_DATOS_PARA_CALCULO:
        resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'SS_Techo_Dias': np.nan,
            'SS_Optimo_Dias': np.nan
        })
        continue

    mu_d = ultimos_18.mean()        # Demanda promedio mensual
    sigma_d = ultimos_18.std()      # Desviación demanda mensual

    if pd.isna(sigma_d) or sigma_d == 0:
        sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

    # --- 🔹 Lead Time: en DÍAS (convertir a meses dividiendo para 30) ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_vals_dias = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_vals_dias) > 0:
            mu_L_dias = lt_vals_dias.mean()      # Promedio en días
            sigma_L_dias = lt_vals_dias.std()    # Desviación en días
            mu_L = mu_L_dias / 30                # Convertir a meses
            sigma_L = sigma_L_dias / 30          # Desviación en meses
        else:
            mu_L = 5 / 30
            sigma_L = 1 / 30  # desviación mínima razonable
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando 5 días.")
        mu_L = 5 / 30
        sigma_L = 1 / 30

    if pd.isna(sigma_L) or sigma_L == 0:
        sigma_L = 0.1 * mu_L if mu_L > 0 else 0.1 / 30

    # --- 🔹 Fórmula completa del Stock de Seguridad (en unidades) ---
    try:
        SS_unidades = Z * np.sqrt(
            (mu_L + R_meses) * sigma_d**2 +   # variabilidad demanda
            (sigma_L**2) * mu_d**2            # variabilidad lead time
        )
    except Exception as e:
        SS_unidades = 0.1

    # --- 🔹 Convertir SS a DÍAS ---
    if mu_d > 0:
        SS_dias = (SS_unidades / mu_d) * 30  # porque mu_d es mensual → (unidades / unidades/mes) = meses → *30 = días
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    # --- 🔹 Stock de Seguridad Techo (en días) ---
    L_dias = mu_L * 30  # volver a días
    SS_techo_dias = SS_dias + L_dias + R_dias
    SS_techo_dias = round(SS_techo_dias, 2)

    # --- 🔹 Stock Óptimo en Días ---
    SS_optimo_dias = SS_dias + (L_dias + R_dias) / 2
    SS_optimo_dias = round(SS_optimo_dias, 2)

    # Guardar resultados
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'SS_Techo_Dias': SS_techo_dias,
        'SS_Optimo_Dias': SS_optimo_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Resultados finales (Lead Time en días → convertido a meses dividiendo entre 30):")
print(resultado_df)

# Opcional: exportar
# resultado_df.to_excel("Stock_Seguridad_Final_Final.xlsx", index=False)

/tmp/ipython-input-3911092054.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24132.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ventas_imp[i] = max(0, estimado)



✅ Resultados finales (Lead Time en días → convertido a meses dividiendo entre 30):
         ITEM  Stock_Seguridad_Dias  SS_Techo_Dias  SS_Optimo_Dias
0      ITEM 1                  4.65           9.62            7.13
1      ITEM 2                  8.50          21.50           15.00
2      ITEM 3                  3.16           7.03            5.09
3      ITEM 4                  6.47          19.92           13.20
4      ITEM 5                  6.17          17.66           11.91
..        ...                   ...            ...             ...
655  ITEM 656                  4.74          11.64            8.19
656  ITEM 657                  4.62          12.30            8.46
657  ITEM 658                  5.49          14.84           10.17
658  ITEM 659                  5.73          16.32           11.02
659  ITEM 660                  5.37          14.86           10.11

[660 rows x 4 columns]
